## 1. 개요 📖

**VectorStore 지원 검색기(VectorStore-backed Retriever)** 는 벡터 저장소를 활용하여 문서를 검색하는 리트리버입니다.

### 🎯 주요 특징

- **🔍 유사도 검색**: 벡터 저장소에 구현된 **유사도 검색(similarity search)** 을 사용
- **🎲 MMR 지원**: **Maximum Marginal Relevance** 알고리즘을 통한 다양성 있는 검색
- **⚡ 빠른 검색**: 벡터 임베딩을 이용한 고속 의미 검색
- **🎛️ 유연한 설정**: 검색 개수, 임계값 등 다양한 매개변수 조정 가능

### 💡 동작 원리

1. **📝 문서 입력** → 텍스트를 벡터로 변환하여 저장
2. **❓ 질문 입력** → 질문을 벡터로 변환  
3. **🔍 유사도 계산** → 저장된 문서 벡터들과 유사도 비교
4. **📋 결과 반환** → 가장 관련성 높은 문서들을 순서대로 반환

이제 실제 코드를 통해 VectorStore 기반 검색기를 단계별로 구현해보겠습니다! 🚀

# 📚 벡터스토어 기반 검색기(VectorStore-backed Retriever)

## 📋 목차

1. [📖 개요](#1-개요)
2. [🛠️ 환경 설정](#2-환경-설정) 
3. [📊 VectorStore 생성](#3-vectorstore-생성)
4. [🔍 VectorStoreRetriever 초기화](#4-vectorstoreretriever-초기화)
5. [⚡ Retriever의 invoke() 메서드](#5-retriever의-invoke-메서드)
6. [🎯 Max Marginal Relevance (MMR)](#6-max-marginal-relevance-mmr)
7. [📈 유사도 점수 임계값 검색](#7-유사도-점수-임계값-검색)
8. [🔢 top_k 설정](#8-top_k-설정)
9. [⚙️ 동적 설정(Configurable)](#9-동적-설정configurable)
10. [🔧 다른 임베딩 모델 사용 사례](#10-다른-임베딩-모델-사용-사례)

---

## 2. 환경 설정 🛠️

VectorStore 기반 검색기를 사용하기 전에 필요한 라이브러리와 설정을 준비하겠습니다.

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## 3. VectorStore 생성 📊

이제 문서를 벡터로 변환하여 저장할 VectorStore를 생성해보겠습니다. FAISS와 OpenAI 임베딩을 사용하겠습니다.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 텍스트 파일을 로드합니다
loader = TextLoader("./data/appendix-keywords.txt")

# 문서를 로드합니다
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다
# chunk_size: 각 청크의 최대 크기 (300자), chunk_overlap: 청크 간 중복 문자 수 (0자)
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 로드된 문서를 설정된 크기로 분할합니다
split_docs = text_splitter.split_documents(documents)

# OpenAI 임베딩 모델을 생성합니다 (text-embedding-3-small 모델 사용)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 분할된 문서와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다
db = FAISS.from_documents(split_docs, embeddings)

## 4. VectorStoreRetriever 초기화 🔍

### 4.1 as_retriever 메서드란? 🤔

`as_retriever` 메서드는 VectorStore 객체를 **검색기(Retriever)** 로 변환하는 핵심 메서드입니다. 

마치 **도서관을 검색 시스템으로 바꾸는 것**과 같습니다:
- **📚 도서관 (VectorStore)**: 책들이 정리되어 있는 저장소
- **🔍 검색 시스템 (Retriever)**: 원하는 책을 빠르게 찾아주는 도구

### 📋 주요 매개변수들

#### 🔍 **search_type** (검색 유형)
- **`similarity`**: 유사도 기반 검색 (기본값)
- **`mmr`**: Maximum Marginal Relevance (다양성 고려)  
- **`similarity_score_threshold`**: 임계값 기반 검색

#### ⚙️ **search_kwargs** (검색 옵션)
- **`k`**: 반환할 문서 수 (기본값: 4)
- **`score_threshold`**: 최소 유사도 임계값 (0.0~1.0)
- **`fetch_k`**: MMR에서 가져올 후보 문서 수 (기본값: 20)
- **`lambda_mult`**: MMR 다양성 조절 (0: 다양성 우선, 1: 유사도 우선)

### ⚠️ 사용 시 주의사항

- `search_type`과 `search_kwargs`의 조합이 중요합니다
- MMR 사용 시 `fetch_k` ≥ `k` 여야 합니다  
- `score_threshold`가 너무 높으면 검색 결과가 없을 수 있습니다
- 메타데이터 필터링 시 데이터 구조를 정확히 파악해야 합니다

In [ ]:
# VectorStore를 Retriever로 변환합니다 (기본 설정: 유사도 검색, k=4)
retriever = db.as_retriever()

## 5. Retriever의 invoke() 메서드 ⚡

### 5.1 invoke()가 무엇인가요? 🤔

`invoke()` 메서드는 실제로 **검색을 실행하는 핵심 메서드**입니다. 

**🕵️ 탐정의 수사 과정**에 비유하면:
1. **🔍 단서 입력**: 질문(쿼리)을 입력
2. **📋 증거 수집**: 벡터 데이터베이스에서 관련 문서 검색
3. **📊 분석**: 유사도 점수 계산 
4. **📝 보고서**: 가장 관련성 높은 문서들을 순서대로 반환

### 📋 매개변수 설명

- **`input`**: 검색할 쿼리 문자열 (필수)
- **`config`**: 검색 설정 (선택사항, 동적 설정에 사용)
- **`**kwargs`**: 추가 검색 옵션

### 🎯 반환값

**`List[Document]`** 형태로 관련 문서 목록을 반환합니다. 각 Document 객체는:
- **`page_content`**: 문서의 실제 내용
- **`metadata`**: 문서의 메타데이터 (출처, 페이지 번호 등)

실제 예시를 통해 invoke() 메서드가 어떻게 동작하는지 확인해보겠습니다! 🚀

In [ ]:
# "임베딩"과 관련된 문서를 검색합니다
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

# 검색된 각 문서의 내용을 출력합니다
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

## 6. Max Marginal Relevance (MMR) 🎯

### 6.1 MMR이 무엇인가요? 🤔

**MMR(Maximal Marginal Relevance)** 은 검색 결과의 **중복을 방지하고 다양성을 높이는** 똑똑한 알고리즘입니다.

### 🍕 피자 주문으로 이해하기

친구들과 피자를 주문한다고 생각해보세요:

#### 🚫 **일반적인 유사도 검색** (MMR 없이)
```
검색어: "맛있는 피자"
결과: 페퍼로니, 페퍼로니 스페셜, 페퍼로니 디럭스, 더블 페퍼로니
```
→ **문제점**: 모두 페퍼로니 계열로 비슷함 😞

#### ✅ **MMR 검색** (다양성 고려)
```
검색어: "맛있는 피자"  
결과: 페퍼로니, 마르게리타, 하와이안, 불고기
```
→ **장점**: 다양한 종류의 맛있는 피자! 😋

### 🎛️ MMR의 핵심 매개변수

#### **`lambda_mult`** (다양성 조절 다이얼)
- **`0.0`**: "**다양성 우선**" - 서로 다른 것들을 많이
- **`0.5`**: "**균형**" - 관련성과 다양성의 적절한 조화  
- **`1.0`**: "**관련성 우선**" - 가장 유사한 것들만

#### **`fetch_k`** vs **`k`**
- **`fetch_k`**: 후보군 개수 (예: 20개의 후보 중에서)
- **`k`**: 최종 선택 개수 (예: 최종 2개 선택)

### 💡 언제 MMR을 사용할까?

✅ **긴 문서에서 다양한 관점이 필요할 때**  
✅ **요약이나 FAQ에서 중복 방지가 중요할 때**  
✅ **창의적인 아이디어나 다각도 분석이 필요할 때**

이제 실제 코드로 MMR이 어떻게 다양한 결과를 제공하는지 확인해보겠습니다! 🚀

### 6.2 MMR 매개변수 설정 ⚙️

MMR 검색을 위한 주요 매개변수들을 설정하는 방법입니다:

- **`search_type="mmr"`**: MMR 알고리즘 사용 설정
- **`k=2`**: 최종 반환할 문서 개수 (2개)
- **`fetch_k=10`**: 후보군에서 가져올 문서 개수 (10개 중에서 선택) 
- **`lambda_mult=0.6`**: 다양성 조절 (0.6 = 관련성 60%, 다양성 40%)

In [ ]:
# MMR(Maximal Marginal Relevance) 검색 방식으로 Retriever 설정
retriever = db.as_retriever(
    search_type="mmr",  # MMR 알고리즘 사용
    search_kwargs={
        "k": 2,            # 최종 반환할 문서 수
        "fetch_k": 10,     # 후보군 문서 수 (이 중에서 k개 선택)
        "lambda_mult": 0.6 # 다양성 조절 (0=다양성우선, 1=관련성우선)
    }
)

# "임베딩"과 관련된 문서를 MMR 방식으로 검색합니다
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

# 검색된 문서들을 출력 (다양성을 고려한 결과)
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

## 7. 유사도 점수 임계값 검색 📈  

### 7.1 임계값 검색이란? 🎯

**유사도 점수 임계값 검색**은 **"충분히 관련성이 있는 문서만 가져오기"** 위한 방법입니다.

### 🎓 대학교 입학 성적으로 이해하기

대학교 입학에서 **최소 커트라인**을 정하는 것과 같습니다:

#### 📊 **일반 검색** (임계값 없음)
```
지원자: A(95점), B(87점), C(65점), D(23점)
결과: 모든 지원자 합격 (점수와 무관하게 상위 4명)
```
→ **문제점**: 관련성이 낮은 결과도 포함 😞

#### ✅ **임계값 검색** (커트라인 80점)
```
지원자: A(95점), B(87점), C(65점), D(23점)  
결과: A, B만 합격 (80점 이상만)
```
→ **장점**: 기준에 맞는 결과만 선별! 🎯

### 🎛️ 임계값 설정 가이드

#### **임계값 범위**
- **`0.0`**: 모든 문서 허용 (관련성 무관)
- **`0.5`**: 보통 수준의 관련성 요구  
- **`0.8`**: 높은 관련성 요구
- **`0.95`**: 매우 높은 관련성 요구

#### ⚠️ **주의사항**
- 임계값이 너무 높으면 **검색 결과가 없을 수 있음**
- 데이터의 품질과 임베딩 모델에 따라 적절한 값이 달라짐
- 보통 **0.7~0.8** 정도가 실용적인 범위

### 💡 언제 사용할까?

✅ **정확한 답변이 중요한 QA 시스템**  
✅ **품질 관리가 필요한 검색 시스템**  
✅ **관련 없는 정보를 걸러내야 할 때**

### 7.2 임계값 검색 설정 방법 ⚙️

임계값 기반 검색을 설정하는 방법입니다:

- **`search_type="similarity_score_threshold"`**: 임계값 기반 검색 활성화
- **`score_threshold=0.8`**: 유사도 점수가 0.8 이상인 문서만 반환

이렇게 설정하면 **관련성이 떨어지는 문서는 자동으로 걸러지고**, 높은 품질의 검색 결과만 얻을 수 있습니다! 🎯

In [ ]:
# 유사도 점수 임계값 기반 검색으로 Retriever 설정
retriever = db.as_retriever(
    # 검색 유형을 임계값 기반으로 설정
    search_type="similarity_score_threshold",
    # 유사도 점수가 0.8 이상인 문서만 반환하도록 임계값 설정
    search_kwargs={"score_threshold": 0.8},
)

# "Word2Vec"과 관련된 문서를 임계값 기준으로 검색합니다
for doc in retriever.invoke("Word2Vec 은 무엇인가요?"):
    print(doc.page_content)
    print("=========================================================")

## 8. top_k 설정 🔢

### 8.1 top_k가 무엇인가요? 🤔

**`k`** 매개변수는 검색 결과에서 **몇 개의 문서를 반환할지**를 결정하는 중요한 설정입니다.

### 🎵 음악 차트로 이해하기

음악 차트에서 순위를 정하는 것과 같습니다:

- **`k=1`**: "**오늘의 1위 곡**" - 가장 관련성 높은 1개만
- **`k=3`**: "**TOP 3**" - 상위 3개 곡  
- **`k=10`**: "**TOP 10**" - 인기 상위 10개 곡

### 📊 k 값 선택 가이드

#### **적은 k 값 (1~3)**
✅ **정확한 정보**를 원할 때  
✅ **빠른 처리**가 필요할 때  
✅ **토큰 사용량**을 줄이고 싶을 때

#### **많은 k 값 (5~10+)**  
✅ **다양한 관점**의 정보가 필요할 때  
✅ **포괄적인 답변**을 만들고 싶을 때  
✅ **문서가 짧아서** 더 많은 컨텍스트가 필요할 때

### 💡 실무 팁

- **QA 시스템**: 보통 `k=3~5`가 적절
- **요약**: `k=5~10`으로 충분한 정보 확보
- **창의적 작업**: `k=7~15`로 다양한 아이디어 수집

이제 `k=1`로 설정하여 **가장 관련성 높은 문서 1개만** 가져오는 예시를 확인해보겠습니다! 🎯

### 8.2 k=1 설정 실습 ⚙️

`search_kwargs`에서 `k` 값을 **1로 설정**하여 가장 관련성 높은 문서 **1개만** 반환하도록 하겠습니다.

In [ ]:
# 반환할 문서 개수를 1개로 제한하여 Retriever 설정
retriever = db.as_retriever(search_kwargs={"k": 1})

# "임베딩"과 관련된 문서를 검색 (최대 1개만 반환)
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

# 검색된 문서 출력 (1개만 출력됨)
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

## 9. 동적 설정(Configurable) ⚙️

### 9.1 동적 설정이란? 🔄

**동적 설정**은 하나의 Retriever를 만들어두고, **실행할 때마다 다른 검색 옵션을 적용**할 수 있는 유연한 기능입니다.

### 🎮 게임 캐릭터 설정으로 이해하기

게임에서 캐릭터를 만들 때를 생각해보세요:

#### 🚫 **기존 방식** (정적 설정)
```
전사_캐릭터 = Character(공격력=100, 방어력=50)
법사_캐릭터 = Character(공격력=50, 방어력=20)  
궁수_캐릭터 = Character(공격력=75, 방어력=30)
```
→ **문제점**: 캐릭터마다 새로 만들어야 함 😞

#### ✅ **동적 설정** (Configurable)
```
캐릭터 = Character().configurable()
# 실행 시점에 설정 적용
캐릭터.play(config={직업: "전사", 공격력: 100})
캐릭터.play(config={직업: "법사", 공격력: 50})  
```
→ **장점**: 하나의 캐릭터로 다양한 플레이 가능! 🎯

### 🔧 ConfigurableField 핵심 개념

#### **ConfigurableField 매개변수**
- **`id`**: 고유 식별자 (설정을 구분하는 키)
- **`name`**: 사용자에게 보여질 이름
- **`description`**: 이 설정이 무엇인지 설명

#### **config 딕셔너리 구조**
```python
config = {
    "configurable": {
        "설정_id": 설정값,
        "다른_설정_id": 다른_설정값
    }
}
```

### 💡 동적 설정의 장점

✅ **코드 재사용**: 하나의 Retriever로 다양한 검색 방식 지원  
✅ **실행 시점 결정**: 상황에 맞게 최적의 검색 전략 선택  
✅ **유연성**: 사용자 요구나 조건에 따라 동적으로 조정  
✅ **메모리 효율성**: 여러 개의 Retriever를 만들 필요 없음

이제 실제 코드로 동적 설정을 구현해보겠습니다! 🚀

In [ ]:
from langchain_core.runnables import ConfigurableField

# 동적으로 설정 가능한 Retriever 생성
retriever = db.as_retriever(search_kwargs={"k": 1}).configurable_fields(
    # 검색 유형을 동적으로 변경 가능하도록 설정
    search_type=ConfigurableField(
        id="search_type",                    # 고유 식별자
        name="Search Type",                  # 사용자에게 보여질 이름
        description="The search type to use", # 이 설정에 대한 설명
    ),
    # 검색 매개변수를 동적으로 변경 가능하도록 설정
    search_kwargs=ConfigurableField(
        id="search_kwargs",                          # 고유 식별자
        name="Search Kwargs",                        # 사용자에게 보여질 이름  
        description="The search kwargs to use",      # 이 설정에 대한 설명
    ),
)

### 9.2 동적 설정 실습 예시들 🧪

이제 같은 Retriever를 사용하여 **실행할 때마다 다른 검색 설정**을 적용해보겠습니다!

In [ ]:
# 예시 1: k=3으로 동적 설정하여 3개 문서 검색
config = {"configurable": {"search_kwargs": {"k": 3}}}

# 같은 retriever를 사용하되, 실행 시점에 k=3 설정 적용
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?", config=config)

# 검색 결과 출력 (3개 문서가 반환됨)
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

In [ ]:
# 예시 2: 임계값 검색으로 동적 설정 변경
config = {
    "configurable": {
        # 검색 유형을 임계값 기반으로 변경
        "search_type": "similarity_score_threshold",
        # 유사도 점수 0.8 이상인 문서만 반환
        "search_kwargs": {
            "score_threshold": 0.8,
        },
    }
}

# 같은 retriever이지만 임계값 검색 방식으로 실행
docs = retriever.invoke("Word2Vec 은 무엇인가요?", config=config)

# 고품질 문서만 출력 (임계값 이상인 것만)
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

In [ ]:
# 예시 3: MMR 검색으로 동적 설정 변경
config = {
    "configurable": {
        # 검색 유형을 MMR로 변경
        "search_type": "mmr",
        # MMR 관련 매개변수 설정
        "search_kwargs": {"k": 2, "fetch_k": 10, "lambda_mult": 0.6},
    }
}

# 같은 retriever이지만 MMR 방식으로 실행 (다양성 고려)
docs = retriever.invoke("Word2Vec 은 무엇인가요?", config=config)

# 다양성을 고려한 검색 결과 출력
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

## 10. 다른 임베딩 모델 사용 사례 🔧

### 10.1 분리된 임베딩 모델이란? 🤔

일반적으로 **쿼리와 문서에 동일한 임베딩 모델**을 사용하지만, 때로는 **각각에 특화된 다른 모델**을 사용하면 더 좋은 성능을 얻을 수 있습니다.

### 🏃‍♂️ 운동선수로 이해하기

마라톤과 단거리 달리기를 생각해보세요:

#### 🏃‍♂️ **일반적인 방식** (같은 선수)
```  
질문: "오늘 날씨가 어때?" (단거리 선수)
문서: "날씨 관련 긴 설명글" (같은 단거리 선수)
```
→ **문제점**: 문서는 긴 내용인데 단거리에 특화된 선수 😞

#### 🏃‍♂️ **특화된 방식** (목적별 선수)
```
질문: "오늘 날씨가 어때?" (단거리 특화 선수)
문서: "날씨 관련 긴 설명글" (장거리 특화 선수)  
```
→ **장점**: 각각의 특성에 맞는 최적의 성능! 🎯

### 🎯 Upstage 임베딩의 특별함

**Upstage**에서는 이런 특화된 모델들을 제공합니다:

#### **쿼리 특화 모델** (`solar-embedding-1-large-query`)
- **짧은 질문**에 최적화
- **검색 의도** 파악에 특화
- **빠른 처리** 속도

#### **문서 특화 모델** (`solar-embedding-1-large-passage`) 
- **긴 문서 내용**에 최적화  
- **정보 압축** 능력 우수
- **맥락 이해** 능력 강화

### 💡 언제 분리된 모델을 사용할까?

✅ **검색 성능이 중요한 상용 서비스**  
✅ **다양한 길이의 텍스트를 처리하는 경우**  
✅ **특정 도메인에 특화된 검색이 필요할 때**  
✅ **높은 정확도가 요구되는 전문 분야**

이제 실제 코드로 분리된 임베딩 모델을 어떻게 사용하는지 확인해보겠습니다! 🚀

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_upstage import UpstageEmbeddings

# TextLoader를 사용하여 텍스트 파일을 로드합니다
loader = TextLoader("./data/appendix-keywords.txt")

# 문서를 로드합니다
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 로드된 문서를 설정된 크기로 분할합니다
split_docs = text_splitter.split_documents(documents)

# 문서(Passage) 전용 Upstage 임베딩 모델을 생성합니다
doc_embedder = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 분할된 문서와 문서 전용 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다
db = FAISS.from_documents(split_docs, doc_embedder)

### 10.2 쿼리 전용 임베딩으로 검색하기 🔍

이제 **쿼리 전용 임베딩 모델**을 사용하여 검색해보겠습니다. 

**핵심 포인트**: 
- **문서 저장**은 `passage` 모델로 
- **검색 쿼리**는 `query` 모델로

이렇게 각각의 특성에 맞는 모델을 사용하면 더 정확한 검색 결과를 얻을 수 있습니다! 🎯

In [ ]:
# 쿼리 전용 Upstage 임베딩 모델을 생성합니다
query_embedder = UpstageEmbeddings(model="solar-embedding-1-large-query")

# 쿼리 문장을 쿼리 전용 모델로 벡터화합니다
query_vector = query_embedder.embed_query("임베딩(Embedding)은 무엇인가요?")

# 벡터 유사도 검색을 수행하여 가장 유사한 2개의 문서를 반환합니다
# (문서는 passage 모델로, 쿼리는 query 모델로 처리된 상태)
db.similarity_search_by_vector(query_vector, k=2)